# Tutorial 01 - Thermal Block

In [ ]:
import typing

In [ ]:
import dolfinx.fem
import gmsh
import multiphenicsx.io
import multiphenicsx.mesh
import numpy as np
import petsc4py
import ufl

In [ ]:
import rbnicsx.backends

## 1. Mesh generation

In [ ]:
mesh_size = 1e-1

In [ ]:
gmsh.initialize()
gmsh.model.add("thermal_block")

In [ ]:
p0 = gmsh.model.geo.addPoint(-1.0, -1.0, 0.0, mesh_size)
p1 = gmsh.model.geo.addPoint(1.0, -1.0, 0.0, mesh_size)
p2 = gmsh.model.geo.addPoint(1.0, 1.0, 0.0, mesh_size)
p3 = gmsh.model.geo.addPoint(-1.0, 1.0, 0.0, mesh_size)
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
outer_boundary = gmsh.model.geo.addCurveLoop([l0, l1, l2, l3])

In [ ]:
p4 = gmsh.model.geo.addPoint(0.0, 0.0, 0.0, mesh_size)
p5 = gmsh.model.geo.addPoint(0.5, 0.0, 0.0, mesh_size)
p6 = gmsh.model.geo.addPoint(-0.5, 0.0, 0.0, mesh_size)
c0 = gmsh.model.geo.addCircleArc(p5, p4, p6)
c1 = gmsh.model.geo.addCircleArc(p6, p4, p5)
interface = gmsh.model.geo.addCurveLoop([c0, c1])

In [ ]:
outer_subdomain = gmsh.model.geo.addPlaneSurface([outer_boundary, interface])
inner_subdomain = gmsh.model.geo.addPlaneSurface([interface])

In [ ]:
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1, l3], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(2, [inner_subdomain], 1)
gmsh.model.addPhysicalGroup(2, [outer_subdomain], 2)
gmsh.model.mesh.generate(2)

In [ ]:
mesh, subdomains, boundaries = multiphenicsx.mesh.gmsh_to_fenicsx(gmsh.model, gdim=2)
gmsh.finalize()

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

## 2. Problem definition

In [ ]:
class Problem(object):
    """Define a linear problem, and solve it with KSP."""

    def __init__(self) -> None:
        # Define function space
        V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
        self._V = V
        # Define trial and test functions
        u = ufl.TrialFunction(V)
        v = ufl.TestFunction(V)
        # Define measures for integration of forms
        dx = ufl.Measure("dx")(subdomain_data=subdomains)
        ds = ufl.Measure("ds")(subdomain_data=boundaries)
        # Define symbolic parameters for use in UFL forms
        mu_symb = rbnicsx.backends.SymbolicParameters(mesh, shape=(2, ))
        self._mu_symb = mu_symb
        # Define bilinear form of the problem
        a = mu_symb[0] * ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(1) + ufl.inner(ufl.grad(u), ufl.grad(v)) * dx(2)
        self._a = a
        self._a_cpp = dolfinx.fem.form(a)
        # Define linear form of the problem
        f = ufl.inner(mu_symb[1], v) * ds(1)
        self._f = f
        self._f_cpp = dolfinx.fem.form(f)
        # Define boundary conditions for the problem
        zero = petsc4py.PETSc.ScalarType(0)
        facets_top = boundaries.indices[boundaries.values == 3]
        bdofs_V_top = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim - 1, facets_top)
        bcs = [dolfinx.fem.dirichletbc(zero, bdofs_V_top, V)]
        self._bcs = bcs

    @property
    def function_space(self) -> dolfinx.fem.FunctionSpace:
        """Return the function space of the problem."""
        return self._V

    @property
    def bilinear_form(self) -> ufl.Form:
        """Return the bilinear form of the problem."""
        return self._a

    @property
    def linear_form(self) -> ufl.Form:
        """Return the linear form of the problem."""
        return self._f

    @property
    def boundary_conditions(self) -> typing.List[dolfinx.fem.DirichletBCMetaClass]:
        """Return the boundary conditions for the problem."""
        return self._bcs

    def _assemble_matrix(self) -> petsc4py.PETSc.Mat:
        """Assemble the left-hand side matrix."""
        A = dolfinx.fem.assemble_matrix(self._a_cpp, bcs=self._bcs)
        A.assemble()
        return A

    def _assemble_vector(self) -> petsc4py.PETSc.Vec:
        """Assemble the right-hand side vector."""
        F = dolfinx.fem.assemble_vector(self._f_cpp)
        F.ghostUpdate(addv=petsc4py.PETSc.InsertMode.ADD, mode=petsc4py.PETSc.ScatterMode.REVERSE)
        dolfinx.fem.set_bc(F, self._bcs)
        return F

    def solve(self, mu: np.typing.NDArray[np.float64]) -> dolfinx.fem.Function:
        """Assign the provided parameters value and solve the problem."""
        self._mu_symb.value[:] = mu
        return self._solve()

    def _solve(self) -> dolfinx.fem.Function:
        """Solve the linear problem with KSP."""
        A = self._assemble_matrix()
        F = self._assemble_vector()
        ksp = petsc4py.PETSc.KSP()
        ksp.create(mesh.comm)
        ksp.setOperators(A)
        ksp.setType("preonly")
        ksp.getPC().setType("lu")
        ksp.getPC().setFactorSolverType("mumps")
        ksp.setFromOptions()
        solution = dolfinx.fem.Function(self._V)
        ksp.solve(F, solution.vector)
        solution.vector.ghostUpdate(
            addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
        return solution

In [ ]:
problem = Problem()

In [ ]:
mu_solve = np.array([8.0, -1.0])
solution = problem.solve(mu_solve)

In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "high fidelity solution")